# Predictive Performance

## On the Explained Sample

### ConceptLIME

In [7]:
import pickle
import numpy as np

with open("Result/CLIME_pred.pickle", "rb") as f:
    clime_pred = pickle.load(f)

clime_nn_pred = np.array([item[-1] for item in clime_pred["target"]])
clime_surrogate_pred = np.array([item[-1] for item in clime_pred["pred"]])

clime_error_list = [abs((x - y) / y) for x, y in zip(clime_surrogate_pred, clime_nn_pred)]

print(f"Maximum Relative Error: {np.max(clime_error_list):.4f}")
print(f"Minimum Relative Error: {np.min(clime_error_list):.4f}")
print(f"Mean Relative Error: {np.mean(clime_error_list):.4f}")
print(f"Standard Deviation: {np.std(clime_error_list):.4f}")

Maximum Relative Error: 0.6654
Minimum Relative Error: 0.0021
Mean Relative Error: 0.1889
Standard Deviation: 0.1444


### LIME

In [15]:
import pickle
import numpy as np

with open("Result/LIME_pred.pickle", "rb") as f:
    lime_pred = pickle.load(f)

lime_nn_pred = lime_pred["target"]
lime_surrogate_pred = lime_pred["pred"]

lime_error_list = [abs((x - y) / y) for x, y in zip(lime_surrogate_pred, lime_nn_pred)]

print(f"Maximum Relative Error: {np.max(lime_error_list):.4f}")
print(f"Minimum Relative Error: {np.min(lime_error_list):.4f}")
print(f"Mean Relative Error: {np.mean(lime_error_list):.4f}")
print(f"Standard Deviation: {np.std(lime_error_list):.4f}")

Maximum Relative Error: 0.4951
Minimum Relative Error: 0.4283
Mean Relative Error: 0.4609
Standard Deviation: 0.0152


### Hypothesis Testing

In [16]:
from scipy.stats import mannwhitneyu

stat, p = mannwhitneyu(clime_error_list, lime_error_list)
print(f"Statistics={stat:.4f}, p={p:.4e}")

Statistics=551.0000, p=1.6123e-27


## On the Training Set of the Surrogate Model

### ConceptLIME

In [1]:
from sklearn.metrics import r2_score
import pickle
import numpy as np

with open("Result/CLIME_pred.pickle", "rb") as f:
    clime_pred = pickle.load(f)

clime_nn_pred = [item for item in clime_pred["target"]]
clime_surrogate_pred = [item for item in clime_pred["pred"]]

clime_r2_list = [r2_score(x, y) for x, y in zip(clime_nn_pred, clime_surrogate_pred)]

print(f"Maximum R2 Score: {np.max(clime_r2_list):.4f}")
print(f"Minimum R2 Score: {np.min(clime_r2_list):.4f}")
print(f"Mean R2 Score: {np.mean(clime_r2_list):.4f}")
print(f"Standard Deviation: {np.std(clime_r2_list):.4f}")

Maximum R2 Score: 0.9966
Minimum R2 Score: 0.3056
Mean R2 Score: 0.8029
Standard Deviation: 0.1388


### LIME

In [2]:
import pickle
import numpy as np

with open("Result/LIME_pred.pickle", "rb") as f:
    lime_pred = pickle.load(f)
    
lime_r2_list = lime_pred["r2"]

print(f"Maximum R2 Score: {np.max(lime_r2_list):.4f}")
print(f"Minimum R2 Score: {np.min(lime_r2_list):.4f}")
print(f"Mean R2 Score: {np.mean(lime_r2_list):.4f}")
print(f"Standard Deviation: {np.std(lime_r2_list):.4f}")

Maximum R2 Score: 0.6322
Minimum R2 Score: 0.5608
Mean R2 Score: 0.5961
Standard Deviation: 0.0153


### Hypothesis Testing

In [3]:
from scipy.stats import mannwhitneyu

stat, p = mannwhitneyu(clime_r2_list, lime_r2_list)
print(f"Statistics={stat:.4f}, p={p:.4e}")

Statistics=9388.0000, p=8.1677e-27


# Fidelity

In [8]:
import pickle
import numpy as np
import matplotlib.pyplot as plt


with open("Result/CLIME_fidel.pickle", "rb") as f:
    clime_fidel = pickle.load(f)


clime_fidel_by_sample = []
for item in clime_fidel:
    clime_fidel_by_sample.append(np.mean([item[key][0] for key in item.keys()]))


max_key = 0
for item in clime_fidel:
    max_key = max(max_key, max(item.keys()))


clime_fidel_by_key = [[] for _ in range(max_key)]
for item in clime_fidel:
    for key in item.keys():
        clime_fidel_by_key[key - 1].append(item[key][0])


clime_fidel_by_sample_and_key = []
for item in clime_fidel:
    for key in item.keys():
        clime_fidel_by_sample_and_key.append(item[key][0])


with open("Result/LIME_fidel.pickle", "rb") as f:
    lime_fidel = pickle.load(f)


lime_fidel_by_sample = []
for item in lime_fidel:
    lime_fidel_by_sample.append(np.mean([item[key][0] for key in item.keys()]))


max_key = 0
for item in lime_fidel:
    max_key = max(max_key, max(item.keys()))


lime_fidel_by_key = [[] for _ in range(10, max_key + 1, 10)]
for item in lime_fidel:
    for key in item.keys():
        lime_fidel_by_key[int(key/10) - 1].append(item[key][0])


lime_fidel_by_sample_and_key = []
for item in lime_fidel:
    for key in item.keys():
        lime_fidel_by_sample_and_key.append(item[key][0])


plt.rcParams["figure.dpi"] = 1200
fig, axs = plt.subplots(2, 3, figsize=(12, 8))


axs[0, 0].axvline(np.mean(clime_fidel_by_sample), color="red", linestyle="dashed", linewidth=1)
axs[0, 0].hist(clime_fidel_by_sample, bins=100)
axs[0, 0].grid(True, linestyle="dashed",axis="y")
axs[0, 0].set_xlim(-0.3, 1)
axs[0, 0].set_title("ConceptLIME\nDistribution Over Samples")
axs[0, 0].set_xlabel("Correlation Values")
axs[0, 0].set_ylabel("Frequency")

axs[0, 1].violinplot(dataset=clime_fidel_by_key, widths=.8, showmeans=True, showextrema=True)
axs[0, 1].grid(True, linestyle="dashed")
axs[0, 1].set_ylim(-0.6, 1)
axs[0, 1].set_title("ConceptLIME\nDistribution On $d$ Values")
axs[0, 1].set_xlabel("Removed Super-pixel Count $d$")
axs[0, 1].set_ylabel("Correlation")

axs[0, 2].hist(clime_fidel_by_sample_and_key, bins=100)
axs[0, 2].axvline(np.mean(clime_fidel_by_sample_and_key), color="red", linestyle="dashed", linewidth=1)
axs[0, 2].grid(True, linestyle="dashed", axis="y")
axs[0, 2].set_xticks(np.array(range(-10, 12, 2))/10)
axs[0, 2].set_xlim(-0.5, 1)
axs[0, 2].set_title("ConceptLIME\nDistribution Over Samples and $d$ Values")
axs[0, 2].set_xlabel("Correlation Values")
axs[0, 2].set_ylabel("Frequency")

axs[1, 0].axvline(np.mean(lime_fidel_by_sample), color="red", linestyle="dashed", linewidth=1)
axs[1, 0].hist(lime_fidel_by_sample, bins=100)
axs[1, 0].grid(True, linestyle="dashed",axis="y")
axs[1, 0].set_xlim(-0.3, 1)
axs[1, 0].set_title("LIME\nDistribution Over Samples")
axs[1, 0].set_xlabel("Correlation Values")
axs[1, 0].set_ylabel("Frequency")

positions = list(range(10, max_key + 1, 10))
axs[1, 1].violinplot(dataset=lime_fidel_by_key, widths=.8, showmeans=True, showextrema=True, positions=positions) # positions=x_ticks
axs[1, 1].grid(True, linestyle="dashed")
axs[1, 1].set_ylim(-0.6, 1)
axs[1, 1].set_title("LIME\nDistribution On $d$ Values")
axs[1, 1].set_xlabel("Removed Super-pixel Count $d$")
axs[1, 1].set_ylabel("Correlation")

axs[1, 2].hist(lime_fidel_by_sample_and_key, bins=100)
axs[1, 2].axvline(np.mean(lime_fidel_by_sample_and_key), color="red", linestyle="dashed", linewidth=1)
axs[1, 2].grid(True, linestyle="dashed", axis="y")
axs[1, 2].set_xticks(np.array(range(-10, 12, 2))/10)
axs[1, 2].set_xlim(-0.5, 1)
axs[1, 2].set_title("LIME\nDistribution Over Samples and $d$ Values")
axs[1, 2].set_xlabel("Correlation Values")
axs[1, 2].set_ylabel("Frequency")


plt.tight_layout()
# plt.show()
plt.savefig(f"Result/fidelity.pdf", bbox_inches = 'tight',pad_inches = 0)

In [10]:
min(clime_fidel_by_sample_and_key)

-0.5003187651707708